In [1]:
pip install HolisticTraceAnalysis

Note: you may need to restart the kernel to use updated packages.


In [4]:
for rank, df in analyzer.t.traces.items():
    print(f"Rank: {rank}, Number of events: {len(df)}")
    print(df["name"].unique())

Rank: 0, Number of events: 6990
[215 109 151 222 256 329 263 248 325 186 129 308 220 174 396   2 380 351
 132  50 290 155  75 383  80  13  31 298 382 201 390 364 384 167  97 279
 188 145  47 150 210 173 184 146 354 367  70 286 175 154 110 267 321 161
 328  39  71  74 221 317 177 389 368 223 287 138  49  82 213  95 239 345
 168 324 399 313   8  48 319 363 301 100 295 358 165 278  22 343 259  15
 332 235 274  78 131 115  38 160  53 285 211 327  72 240 158  57 159 336
 111 280 342 238 208 373 260 254  84  11 143 275 310 258 377 185 209  73
 101  23  41  35 369 272 216  64 162 249 264 365 232 305 108  79 144   4
   5 255 199 391 130  21 385  77 152 181  56 244 276 356  30 147 375  85
  51 341 236 359 312  89  67 371 135 282  28 299 116  17 394  81  40 139
  37 120  68 393 326 398  55  87 253 333  10 357   3 281 214 381  42  18
  61 353 230 302 190 102 153 247 134   7  54 192 386 180 142  16 350 323
 227 118 107 203   1  34 123 182 252 122 334 277 112  25 224 231  88 289
 241 361 320 106 24

In [3]:
from hta.trace_analysis import TraceAnalysis

analyzer = TraceAnalysis(trace_dir="trace_dir")
analyzer.get_temporal_breakdown(visualize=True)

If the trace file does not have the rank specified in it, then add the following snippet key to the json files to use HTA; "distributedInfo": {"rank": 0}. If there are multiple traces files, then each file should have a unique rank value.For now we will default to rank = 0.
Parsed trace_dir\gpt2_trace-checkpoint.json time = 0.18 seconds 
Rounding down ns resolution events due to issue with events overlapping. ts dtype = float64, dur dtype = float64.Please see https://github.com/pytorch/pytorch/pull/122425
Parsed trace_dir\gpt2_trace-checkpoint.json backend=ParserBackend.JSON in 0.42 seconds; current PID:4512
Overall parsing of trace_dir\gpt2_trace-checkpoint.json in 0.46 seconds; current PID:4512
leaving parse_multiple_ranks duration=0.47 seconds
leaving parse_traces duration=0.47 seconds


IndexError: single positional indexer is out-of-bounds

In [10]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.profiler import profile, record_function, ProfilerActivity
import time

# Confirm CUDA is available
assert torch.cuda.is_available(), "CUDA not available"

# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to("cuda")
model.eval()

# Prepare a longer input to increase GPU workload
prompt = "The future of AI is" * 50  # Repeat to make it longer
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
inputs_cuda = {k: v.to("cuda") for k, v in inputs.items()}

# Warm-up to initialize CUDA kernels
with torch.no_grad():
    for _ in range(3):
        _ = model(**inputs_cuda)

# Profile with both CPU and CUDA activities
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    record_shapes=True,
    with_stack=True,
    profile_memory=True
) as prof:
    with record_function("gpt2_inference"):
        _ = model(**inputs_cuda)
        torch.cuda.synchronize()  # Ensure all CUDA ops are completed
        time.sleep(0.1)  # Give profiler time to flush events

# Save trace
prof.export_chrome_trace("gpt2_trace-checkpoint.json")
print("Trace saved to gpt2_trace-checkpoint.json")

Trace saved to gpt2_trace-checkpoint.json


In [9]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.5.1
12.4
True
